## Exploring RAJA and OpenMP

This notebook illustrates how RAJA and OpenMP constructs can work
interactively within Jupyter using a customized Jupyter kernel

[Back to Table of Contents](./00-intro-and-contents.ipynb)


## Usage

<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa"
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <div>
    To run the selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    </div>
</div>

## Notebook setup
You'll need to run the following cell in order for Cling to find
OpenMP and RAJA headers and Libraries

In [ ]:
#pragma cling add_include_path("/opt/conda/include/")
#pragma cling add_library_path("/opt/conda/lib/")
#pragma cling load("libomp")

#pragma cling add_include_path("/home/jovyan/spack/opt/spack/linux-ubuntu22.04-x86_64/gcc-10.4.0/camp-2022.03.2-nh5kkatqzxgiwxusi4ddpyhf2zcqxyow/include/")
#pragma cling add_include_path("/home/jovyan/spack/opt/spack/linux-ubuntu22.04-x86_64/gcc-10.4.0/raja-2022.03.0-f7p4trkfeq4gcmqnt5623haejgcvwlvl/include/")
#pragma cling add_library_path("/home/jovyan/spack/opt/spack/linux-ubuntu22.04-x86_64/gcc-10.4.0/raja-2022.03.0-f7p4trkfeq4gcmqnt5623haejgcvwlvl/lib/")
#pragma cling load("libRAJA")


## Simple OpenMP test program

In [ ]:
#include <omp.h>
#include <iostream>

/**
 * Simple test program that uses OpenMP without guards.  Should only be compiled when OpenMP is enabled.
 */

  #pragma omp parallel
  {
    int thId = omp_get_thread_num();
    int thNum = omp_get_num_threads();
    int thMax = omp_get_max_threads();

    #pragma omp critical
    std::cout <<"\nMy thread id is: " << thId
              <<"\nNum threads is: " << thNum
              <<"\nMax threads is: " << thMax
              << std::endl;
  }


## Sequential PI approximation using Riemann Integral Sum
The following code contains both a C style algorithm and
the RAJA variant using a sequential policy

[Policies](https://raja.readthedocs.io/en/develop/sphinx/user_guide/feature/policies.html)

Various RAJA policies are used for loop kernel execution, scans, reductions, atomics, etc. 
Each policy is a type that is passed to a RAJA template method or class to specialize its behavior. 
Typically, the policy indicates which programming model back-end to use and sometimes provides 
additional information about the execution pattern, such as number of CUDA threads per threadblock,
whether execution is synchronous or asynchronous, etc.

In [ ]:
#include <omp.h>
#include <cstdlib>
#include <iostream>
#include <iomanip>

#include "RAJA/RAJA.hpp"

//
// Define number of subintervals (N) and size of each subinterval (dx) used in
// Riemann integral sum to approximate pi.
//
  const int N = 512 * 512;
  const double dx = 1.0 / double(N);

// Set precision for printing pi
  int prec = 16;
//----------------------------------------------------------------------------//
// C-style sequential variant establishes reference solution to compare with.
//----------------------------------------------------------------------------//
{
  std::cout << "\n Running C-style sequential pi approximation...\n";

  double c_pi = 0.0;

  for (int i = 0; i < N; ++i) {
      double x = (double(i) + 0.5) * dx;
      c_pi += dx / (1.0 + x * x);
  }
  c_pi *= 4.0;

  std::cout << "\tpi = " << std::setprecision(prec)
            << c_pi << std::endl;
}
//----------------------------------------------------------------------------//
// RAJA sequential variant.
//----------------------------------------------------------------------------//
{
  std::cout << "\n Running RAJA sequential pi approximation...\n";

  using EXEC_POL1   = RAJA::seq_exec;
  using REDUCE_POL1 = RAJA::seq_reduce;

  RAJA::ReduceSum< REDUCE_POL1, double > seq_pi(0.0);

  RAJA::forall< EXEC_POL1 >(RAJA::RangeSegment(0, N), [=](int i) {
      double x = (double(i) + 0.5) * dx;
      seq_pi += dx / (1.0 + x * x);
  });
  double seq_pi_val = seq_pi.get() * 4.0;

  std::cout << "\tpi = " << std::setprecision(prec)
            << seq_pi_val << std::endl;
}


## OpenMP based PI approximation using Riemann Integral Sum
The following cell contains C-style code using OpenMP
Note that the output differs slightly from the sequential approximation

In [ ]:
//----------------------------------------------------------------------------//
// C-style OpenMP multithreading variant.
//----------------------------------------------------------------------------//

#if defined(RAJA_ENABLE_OPENMP)
{
  std::cout << "\n Running C-style OpenMP vector addition...\n";

  double c_pi_omp = 0.0;

  #pragma omp parallel for reduction(+:c_pi_omp)
  for (int i = 0; i < N; ++i) {
      double x = (double(i) + 0.5) * dx;
      c_pi_omp += dx / (1.0 + x * x);
  }
  c_pi_omp *= 4.0;

  std::cout << "\tpi = " << std::setprecision(prec)
            << c_pi_omp << std::endl;
}
#endif



## RAJA OpenMP based PI approximation using Riemann Integral Sum
The following cell contains the RAJA variant using the
omp_parallel_for_exec policy. This creates an OpenMP
parallel region and execute with CPU multithreading inside it.
Analgous to the C-style variant above which used
 pragma omp parallel for

Also analagous to the C-style variant, RAJA defines a reduction
variable omp_pi using the omp_reduce policy.

In [ ]:

//----------------------------------------------------------------------------//
// RAJA OpenMP multithreading variant.
//----------------------------------------------------------------------------//

#if defined(RAJA_ENABLE_OPENMP)
{
  std::cout << "\n Running RAJA OpenMP pi approximation...\n";

  using EXEC_POL2   = RAJA::omp_parallel_for_exec;
  using REDUCE_POL2 = RAJA::omp_reduce;

  RAJA::ReduceSum< REDUCE_POL2, double > omp_pi(0.0);

  RAJA::forall< EXEC_POL2 >(RAJA::RangeSegment(0, N), [=](int i) {
      double x = (double(i) + 0.5) * dx;
      omp_pi += dx / (1.0 + x * x);
  });
  double omp_pi_val = omp_pi.get() * 4.0;

  std::cout << "\tpi = " << std::setprecision(prec)
            << omp_pi_val << std::endl;
}
#endif

